In [1]:
import tensorflow as tf
import torch
import torch.nn as nn
from tensorflow.keras.losses import SparseCategoricalCrossentropy
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.decomposition import PCA
from tensorflow.keras import models, layers, regularizers

In [2]:
config = tf.compat.v1.ConfigProto(gpu_options = tf.compat.v1.GPUOptions(per_process_gpu_memory_fraction=0.8))
config.gpu_options.allow_growth = True
session = tf.compat.v1.Session(config=config)
tf.compat.v1.keras.backend.set_session(session)
physical_devices = tf.config.experimental.list_physical_devices('GPU')

print("Num GPUs Available: ", len(physical_devices))


Num GPUs Available:  1


In [3]:
#load the embeddings
loaded_embeddings = np.load("Wood_Embeddings.npy", allow_pickle=True).item()

In [4]:
print(loaded_embeddings.keys())

dict_keys(['heartwood', 'sapwood'])


In [5]:
heartwood_embeddings = loaded_embeddings['heartwood']
sapwood_embeddings = loaded_embeddings['sapwood']

# Verify their shapes or contents if needed
print(f"Heartwood Embeddings Shape: {heartwood_embeddings.shape}")
print(f"Sapwood Embeddings Shape: {sapwood_embeddings.shape}")

Heartwood Embeddings Shape: torch.Size([1024])
Sapwood Embeddings Shape: torch.Size([1024])


In [ ]:
# Projection to match FC weights size
projection_layer = nn.Linear(1024, )  # Adjust output dim to match your FC

sapwood_emb_projected = projection_layer(heartwood_embeddings)
heartwood_emb_projected = projection_layer(heartwood_embeddings)

# Convert PyTorch tensors to NumPy arrays
sapwood_emb_projected = sapwood_emb_projected.detach().cpu().numpy()
heartwood_emb_projected = heartwood_emb_projected.detach().cpu().numpy()

In [ ]:
# Verify their shapes
print(f"Heartwood Embeddings Shape: {sapwood_emb_projected.shape}")
print(f"Sapwood Embeddings Shape: {heartwood_emb_projected.shape}")

Heartwood Embeddings Shape: (128,)
Sapwood Embeddings Shape: (128,)


In [105]:
#LOAD PCA DATA
data_train = np.load('data_train_PCA.npy')
data_test = np.load('data_test_PCA.npy')

train_labels = np.load('Wood_train_labels.npy', allow_pickle= True)
test_labels = np.load('Wood_test_labels.npy', allow_pickle= True)

print(data_train.shape, data_test.shape, train_labels.shape, test_labels.shape )

(264, 32, 32, 199, 1) (132, 32, 32, 199, 1) (264,) (132,)


In [16]:
le= LabelEncoder()
y_train = le.fit_transform(train_labels)

le2= LabelEncoder()
y_test= le2.fit_transform(test_labels)

In [79]:
'''
def augment_data(sample, label):
    
    # Random flip along different axes
    sample = tf.image.random_flip_left_right(sample)
    sample = tf.image.random_flip_up_down(sample)

    # Random rotation
    sample = tf.image.rot90(sample, k=np.random.randint(1, 4))  # k=1 to 3 random rotations of 90°

    # Random intensity scaling (brightness variation)
    sample = sample * tf.random.uniform([], 0.9, 1.1)  # Random scale between 0.9 and 1.1
    
    return sample, label

'''
def augment_data(sample, label):
    
    # Random flip along different axes
    sample = tf.image.random_flip_left_right(sample)
    sample = tf.image.random_flip_up_down(sample)

    # Random rotation
    sample = tf.image.rot90(sample, k=np.random.randint(1, 4))  # k=1 to 3 random rotations of 90°

    # Random intensity scaling (brightness variation)
    sample = sample * tf.random.uniform([], 0.9, 1.1)  # Random scale between 0.9 and 1.1

    # Ensure the shape is correct
    sample = tf.transpose(sample, perm=[0, 1, 2, 3])  # Reorder dimensions

    return sample, label


In [ ]:
'''
def fix_shape(sample, label):
    # Reorder dimensions to match the model's expected input
    sample = tf.transpose(sample, perm=[0, 1, 3, 2])
    
    return sample, label
'''

In [80]:
# Convert data to TensorFlow dataset

train_dataset = tf.data.Dataset.from_tensor_slices((data_train, y_train))
test_dataset = tf.data.Dataset.from_tensor_slices((data_test, y_test))

'''
# Fix train and test datasets
train_dataset = train_dataset.map(fix_shape, num_parallel_calls=tf.data.AUTOTUNE)
test_dataset = test_dataset.map(fix_shape, num_parallel_calls=tf.data.AUTOTUNE)
'''

# Apply augmentation only on training data
train_dataset = train_dataset.map(augment_data, num_parallel_calls=tf.data.AUTOTUNE)

# Shuffle, batch, and prefetch
batch_size = 8

train_dataset = train_dataset.shuffle(buffer_size=100).batch(batch_size).prefetch(tf.data.AUTOTUNE)
test_dataset = test_dataset.batch(batch_size).prefetch(tf.data.AUTOTUNE)

In [82]:
for batch, _ in train_dataset.take(5):
    
    print("Batch shape:", batch.shape)

Batch shape: (8, 32, 32, 199, 1)
Batch shape: (8, 32, 32, 199, 1)
Batch shape: (8, 32, 32, 199, 1)
Batch shape: (8, 32, 32, 199, 1)
Batch shape: (8, 32, 32, 199, 1)


In [83]:
for x, y in train_dataset.take(5):
    
    print(f"Input batch shape: {x.shape}, Label batch shape: {y.shape}")

Input batch shape: (8, 32, 32, 199, 1), Label batch shape: (8,)
Input batch shape: (8, 32, 32, 199, 1), Label batch shape: (8,)
Input batch shape: (8, 32, 32, 199, 1), Label batch shape: (8,)
Input batch shape: (8, 32, 32, 199, 1), Label batch shape: (8,)
Input batch shape: (8, 32, 32, 199, 1), Label batch shape: (8,)


In [84]:
'''
class CustomModel(tf.keras.Model):
    
    def __init__(self, base_model, sapwood_emb, heartwood_emb):
        
        super(CustomModel, self).__init__()
        self.base_model = base_model

        # Pre-computed embeddings for labels
        self.sapwood_emb = tf.constant(sapwood_emb.detach().cpu().numpy(), dtype=tf.float32)
        self.heartwood_emb = tf.constant(heartwood_emb.detach().cpu().numpy(), dtype=tf.float32)


        # Projection layer to align dimensions
        self.projection_layer = tf.keras.layers.Dense(128, activation=None)


    def call(self, inputs):
        
        # Forward pass through CNN
        logits = self.base_model(inputs)  # Shape: (batch_size, 2)

        # Project embeddings to match FC layer
        sapwood_proj = self.projection_layer(self.sapwood_emb)
        heartwood_proj = self.projection_layer(self.heartwood_emb)

        # Extract FC layer weights
        fc_weights = self.base_model.layers[-1].kernel  # Shape: (2, 128)

        # Compute dot products
        sapwood_logits = tf.reduce_sum(fc_weights[0] * sapwood_proj)
        heartwood_logits = tf.reduce_sum(fc_weights[1] * heartwood_proj)

        # Add embedding logits to CNN logits
        logits = logits + tf.stack([sapwood_logits, heartwood_logits])

        return logits
'''

class CustomModel(tf.keras.Model):
    
    def __init__(self, base_model, sapwood_emb, heartwood_emb):
        
        super(CustomModel, self).__init__()
        self.base_model = base_model

        # Pre-computed embeddings for labels
        self.sapwood_emb = tf.constant(sapwood_emb.detach().cpu().numpy(), dtype=tf.float32)
        self.heartwood_emb = tf.constant(heartwood_emb.detach().cpu().numpy(), dtype=tf.float32)

        # Projection layer to align dimensions
        self.projection_layer = tf.keras.layers.Dense(128, activation=None)  # Example size


    def call(self, inputs):
        # Base model output
        logits = self.base_model(inputs)  # Shape: (batch_size, 2)

        # Ensure embeddings are 2D before projection
        sapwood_emb_reshaped = tf.expand_dims(self.sapwood_emb, axis=0)  # Add batch dimension
        sapwood_proj = self.projection_layer(sapwood_emb_reshaped)

        heartwood_emb_reshaped = tf.expand_dims(self.heartwood_emb, axis=0)
        heartwood_proj = self.projection_layer(heartwood_emb_reshaped)

        #print("Sapwood projection shape:", sapwood_proj.shape)
        #print("Heartwood projection shape:", heartwood_proj.shape)
        
        '''
        # Extract FC layer weights
        fc_weights = self.base_model.layers[-1].kernel  # Shape: (2, 128)

        # Compute dot products
        sapwood_logits = tf.reduce_sum(fc_weights[0] * sapwood_proj)
        heartwood_logits = tf.reduce_sum(fc_weights[1] * heartwood_proj)
        
        fc_weights[0].shape

        # Add embedding logits to CNN logits
        logits = logits + tf.stack([sapwood_logits, heartwood_logits])
        '''

        # Optionally, combine outputs or perform other operations
        return logits, sapwood_proj, heartwood_proj


In [88]:
# Base model (CNN)
input_shape = (32, 32, optimal_components, 1)

cnn_model = models.Sequential([
    
    layers.Conv3D(32, kernel_size=(3, 3, 3), activation='relu',
                  kernel_regularizer=regularizers.l2(0.01), input_shape=input_shape),
    
    layers.AveragePooling3D(pool_size=(2, 2, 2)),
    
    layers.Conv3D(64, kernel_size=(3, 3, 3), activation='relu', kernel_regularizer=regularizers.l2(0.001)),
    
    layers.Conv3D(128, kernel_size=(3, 3, 3), activation='relu', kernel_regularizer=regularizers.l2(0.0001)),
    
    layers.AveragePooling3D(pool_size=(2, 2, 2)),
    layers.Flatten(),
    
    layers.Dense(128, activation='softmax', kernel_regularizer=regularizers.l2(0.001)),
    #layers.Dropout(0.5),
    
    #layers.Dense(64, activation='relu', kernel_regularizer=regularizers.l2(0.0001)),
    
    #layers.Dense(2, activation=None)  # Remove activation for logits
])

# Custom model
model = CustomModel(base_model = cnn_model, sapwood_emb = sapwood_emb_projected, heartwood_emb = heartwood_emb_projected)

In [89]:
x = tf.random.normal((8, 32, 32, 199, 1))  # Batch of 8
output = model(x)

print("Base model output shape:", output[0].shape)  # Output from base model
print("Sapwood projection shape:", output[1].shape)  # Should be (1, 128)
print("Heartwood projection shape:", output[2].shape)  # Should be (1, 128)

Base model output shape: (8, 128)
Sapwood projection shape: (1, 128)
Heartwood projection shape: (1, 128)


In [90]:
model.compile(optimizer='adam', loss= SparseCategoricalCrossentropy(from_logits=True), metrics=['accuracy'])

# Define the callbacks
checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath = 'best_model_test.h5',    
    monitor = 'accuracy',          
    save_best_only = True,              
    save_weights_only = False,             
    mode = 'max',                          
    verbose = 1
)

early_stopping_callback = tf.keras.callbacks.EarlyStopping(
    monitor = 'loss',
    patience = 15,
    restore_best_weights = True
)

history = model.fit(
    
    train_dataset,
    epochs=100,
    validation_data=test_dataset,
    callbacks=[checkpoint_callback, early_stopping_callback]
)

Epoch 1/100


c:\Users\rafin\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\backend.py:5582: UserWarning: "`sparse_categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a Softmax activation and thus does not represent logits. Was this intended?
  output, from_logits = _get_logits(


InvalidArgumentError: Graph execution error:

Detected at node 'sparse_categorical_crossentropy_1/SparseSoftmaxCrossEntropyWithLogits/SparseSoftmaxCrossEntropyWithLogits' defined at (most recent call last):
    File "c:\Users\rafin\AppData\Local\Programs\Python\Python310\lib\runpy.py", line 196, in _run_module_as_main
      return _run_code(code, main_globals, None,
    File "c:\Users\rafin\AppData\Local\Programs\Python\Python310\lib\runpy.py", line 86, in _run_code
      exec(code, run_globals)
    File "C:\Users\rafin\AppData\Roaming\Python\Python310\site-packages\ipykernel_launcher.py", line 17, in <module>
      app.launch_new_instance()
    File "C:\Users\rafin\AppData\Roaming\Python\Python310\site-packages\traitlets\config\application.py", line 1075, in launch_instance
      app.start()
    File "C:\Users\rafin\AppData\Roaming\Python\Python310\site-packages\ipykernel\kernelapp.py", line 739, in start
      self.io_loop.start()
    File "C:\Users\rafin\AppData\Roaming\Python\Pytho

In [ ]:
# Plot training & validation loss
plt.figure(figsize=(12, 4))
plt.subplot(1, 2, 1)
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Loss over Epochs')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

# Plot training & validation accuracy
plt.subplot(1, 2, 2)
plt.plot(history.history['accuracy'], label='Train Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.title('Accuracy over Epochs')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()

plt.show()

In [ ]:
# Get true labels and predictions
test_labels = np.concatenate([y for x, y in test_dataset], axis=0)
predictions = np.argmax(model.predict(test_dataset), axis=-1)

In [ ]:
# Classification report
print("Classification Report:")
print(classification_report(test_labels, predictions))

In [ ]:
# Confusion matrix
conf_matrix = confusion_matrix(test_labels, predictions)

plt.figure(figsize=(8, 6))
sns.heatmap(conf_matrix, annot=True, fmt="d", cmap="Blues", xticklabels=[str(i) for i in range(2)], yticklabels=[str(i) for i in range(2)])

plt.title("Confusion Matrix")
plt.xlabel("Predicted Label")
plt.ylabel("True Label")

plt.show()